In [7]:
import sys
import os

from data import read_datastore

BASE_PATH = os.getenv('PROJECTPATH')
# Get the path of the current notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# Add the src directory to the system path
src_path = os.path.join(notebook_dir, BASE_PATH, 'src')
sys.path.append(src_path)

In [8]:
import great_expectations as gx
import pandas as pd
import numpy as np
from data import preprocess_data

In [9]:
df, _version = read_datastore()
X, y = preprocess_data(df)
X.head()

/home/user/project/MLOps-for-music-popularity-prediction/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_function_transformer.py:303: UserWarning: When set_output is configured to be 'pandas', func should return a pandas DataFrame to follow the set_output API  or feature_names_out should be defined.
  warnings.warn(warn_msg.format("pandas"))



,genres__adult standards,genres__alternative rock,genres__ambient,genres__anime,genres__baroque,genres__beats,genres__blues,genres__calming instrumental,genres__ccm,genres__classical,...,dates__weekday_sin,dates__weekday_cos,dates__year,text__album_name_length,text__name_length,text__album_name_word_count,text__name_word_count,int__chart,int__mode,bool__explicit
0,False,False,False,False,False,False,False,False,False,False,...,0.974928,-0.222521,2018,15,52,2,10,0,1,False
1,False,False,False,False,False,False,False,False,False,False,...,0.781831,0.623490,1996,15,25,4,5,1,1,False
2,False,False,False,False,False,False,False,False,False,False,...,0.000000,1.000000,2014,13,26,3,6,0,0,False
3,False,False,False,False,False,False,False,False,False,False,...,0.974928,-0.222521,2001,16,21,3,4,0,1,False
4,False,False,False,False,False,False,False,False,False,False,...,-0.433884,-0.900969,2014,47,38,7,8,0,1,False


In [10]:
BASE_PATH

'/home/user/project/MLOps-for-music-popularity-prediction'

In [11]:
import great_expectations as gx
from great_expectations.data_context import FileDataContext
context = gx.get_context(context_root_dir=f"{BASE_PATH}/services/gx")
context

FileDataContext loading fluent config
Loading 'datasources' ->
[{'assets': [...], 'name': 'sample', 'type': 'pandas'},
 {'assets': [...], 'name': 'sample_data', 'type': 'pandas'},
 {'assets': [...], 'name': 'features', 'type': 'pandas'}]
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
Loaded 'sample' from fluent config
Loaded 'sample_data' from fluent config
Loaded 'features' from fluent config


{
  "anonymous_usage_statistics": {
    "enabled": true,
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "explicit_id": true,
    "data_context_id": "658ab671-7357-4b71-8760-718815eecf13",
    "explicit_url": false
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 3.0,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {},
  "evaluation_parameter_store_name": "evaluation_parameter_store",
  "expectations_store_name": "expectations_store",
  "fluent_datasources": {
    "sample": {
      "type": "pandas",
      "assets":

In [12]:
data_asset = context.get_datasource("features").get_asset("features_dataframe")
batch_request = data_asset.build_batch_request(dataframe=X)

context.add_or_update_expectation_suite("features_expectations")

context.list_expectation_suites()

[ExpectationSuiteIdentifier::features_expectations,
 ExpectationSuiteIdentifier::expectation_suite]

In [13]:
for i in X.columns:
    print(i, X[i].nunique())

genres__adult standards 2
genres__alternative rock 2
genres__ambient 2
genres__anime 2
genres__baroque 2
genres__beats 2
genres__blues 2
genres__calming instrumental 2
genres__ccm 2
genres__classical 2
genres__country 2
genres__disco 2
genres__early music 2
genres__early romantic era 2
genres__easy listening 2
genres__edm 2
genres__electro 2
genres__folk 2
genres__funk 2
genres__german romanticism 2
genres__grunge 2
genres__hardcore 2
genres__hip hop 2
genres__house 2
genres__indietronica 2
genres__jazz 2
genres__late romantic era 2
genres__lo-fi 2
genres__lounge 2
genres__mellow gold 2
genres__metal 2
genres__musica mexicana 2
genres__neo mellow 2
genres__new romantic 2
genres__new wave 2
genres__pop 2
genres__post-romantic era 2
genres__pov: indie 2
genres__punk 2
genres__r&b 2
genres__rap 2
genres__reggae 2
genres__reggaeton 2
genres__rock 2
genres__singer-songwriter 2
genres__soul 2
genres__soundtrack 2
genres__stomp and holler 2
genres__swing 2
genres__synthpop 2
genres__techno 2


In [14]:
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="features_expectations"
)


for column in X.columns:
    # All columns should not have null values
    validator.expect_column_values_to_not_be_null(column)
    if column.startswith("genres") or column.startswith("multilabel") or column.startswith("categorical") or column.startswith("bool"):
        # Expect the column to be boolean (True or False)
        validator.expect_column_values_to_be_in_set(column, [True, False])

    elif column.endswith("_sin") or column.endswith("_cos"):
        validator.expect_column_values_to_be_between(column, min_value=-1, max_value=1)
        validator.expect_column_values_to_be_of_type(column, "float")

    elif column.endswith("year") or column.endswith("_length") or column.endswith("_count") or column.startswith("int"):
        # All columns should be positive
        validator.expect_column_min_to_be_between(column, min_value=0, max_value=None)
        # Column must be of type int
        validator.expect_column_values_to_be_of_type(column, "int")

    elif column.startswith("normal") or column.startswith("uniform"):
        # Column must be of type float
        validator.expect_column_values_to_be_of_type(column, "float")

/home/user/project/MLOps-for-music-popularity-prediction/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

In [15]:
validator.save_expectation_suite(discard_failed_expectations=False)

	587 expectation(s) included in expectation_suite.


In [16]:
checkpoint = context.add_or_update_checkpoint(
    name = "features_test_checkpoint",
    validations = [
        {
            "batch_request": batch.batch_request,
            "expectation_suite_name": "features_expectations"
        } for batch in data_asset.get_batch_list_from_batch_request(batch_request)
    ]
)

In [17]:
# Print failed expectations
res = checkpoint.run()
res

	587 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/2785 [00:00<?, ?it/s]

{
  "run_id": {
    "run_name": null,
    "run_time": "2024-07-15T19:29:03.906448+03:00"
  },
  "run_results": {
    "ValidationResultIdentifier::features_expectations/__none__/20240715T162903.906448Z/features-features_dataframe": {
      "validation_result": {
        "success": true,
        "results": [
          {
            "success": true,
            "expectation_config": {
              "expectation_type": "expect_column_values_to_not_be_null",
              "kwargs": {
                "column": "genres__adult standards",
                "batch_id": "features-features_dataframe"
              },
              "meta": {}
            },
            "result": {
              "element_count": 179940,
              "unexpected_count": 0,
              "unexpected_percent": 0.0,
              "partial_unexpected_list": [],
              "partial_unexpected_counts": [],
              "partial_unexpected_index_list": []
            },
            "meta": {},
            "exception_inf